In [1]:
import numpy as np

#状态转移概率矩阵
P = np.array([
    [0.9, 0.1, 0.0, 0.0, 0.0, 0.0],
    [0.5, 0.0, 0.5, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.6, 0.0, 0.4],
    [0.0, 0.0, 0.0, 0.0, 0.3, 0.7],
    [0.0, 0.2, 0.3, 0.5, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
])

#到达每一个状态的奖励
R = np.array([-1, -2, -2, 10, 1, 0])

$G_t$为一个轨迹的discounted return
$
\begin{aligned}
G_t &= R_{t+1}+\gamma R_{t+2} + \gamma^2 R_{t+3} + …… \\
	&= R_{t+1}+\gamma (R_{t+2} + \gamma R_{t+3} + ……) \\
	&= R_{t+1} + \gamma G_{t+1}
\end{aligned}
$

In [2]:
#给定一条序列,计算回报,计算的就是discounted return
def value_by_chain(chain):
    s = 0
    for i, c in enumerate(chain):
        #给每一步的反馈做一个系数,随着步数往前衰减
        s += R[c] * 0.5**i

    #最终的反馈是所有步数衰减后的求和
    return s
value_by_chain(np.array([0, 1, 2, 5]))

-2.5

贝尔曼公式为
- 代数形式
    $
    \begin{aligned}
    v_\pi(s)
    &= \mathbb{E}[R_{t+1}|S_t=s] +\gamma\mathbb{E}[G_{t+1}|S_t=s], \\
    &= \underbrace{\sum_{a}\pi(a|s)\sum_r p(r|s,a)r}_{当前奖励的平均值} +\underbrace{\gamma \sum_a \pi(a|s)\sum_{s'}p(s'|s,a)v_{\pi}(s')}_{未来奖励的平均值}\\
    &= 	\sum_{a}\pi(a|s) \left[ \sum_{r}p(r|s,a)r+\gamma\sum_{s'}p(s’|s,a)v_{\pi}(s') \right], \forall s\in S.

    \end{aligned}
    $
    描述了不同状态值之间的关系，对于所有状态都成立，即一个状态对应一个式子

- 矩阵形式
    本次实验的贝尔曼公式（矩阵形式）
    $
        v_{\pi}(s_i)=r_{\pi}(s_i)+\gamma \sum_{s_j}p_{\pi}(s_j|s_i)v_{\pi}(s_j)
    $
    $
    \underbrace{v_{\pi}(s)}_{value[i]} = \underbrace{r(s)}_{R[i]} + \underbrace{γP_{\pi}v_{\pi}(s')}_{γ=0.5,\ 0.5 * P[i].dot(value)}
    $

In [4]:
#梯度下降法计算贝尔曼矩阵,近似值
# 由于对所有状态都成立，所以要遍历所有状态
def get_bellman():
    #初始化values
    value = np.ones([6])

    for _ in range(200):
        for i in range(6):
            #每一行的概率和它对应的value相乘，乘以gamma，然后和奖励相加
            #反复计算，就收敛到了贝尔曼方程矩阵
            value[i] = R[i] + 0.5 * P[i].dot(value)

    return value
get_bellman()

array([-2.01950168e+00, -2.21451846e+00,  1.16142785e+00,  1.05380928e+01,
        3.58728554e+00,  6.22301528e-61])

In [5]:
#解析解贝尔曼方程矩阵，真实值
def get_bellman():
    mat = np.eye(*P.shape)
    mat -= 0.5 * P
    mat = np.linalg.inv(mat)

    return mat.dot(R)


get_bellman()

array([-2.01950168, -2.21451846,  1.16142785, 10.53809283,  3.58728554,
        0.        ])